<a href="https://colab.research.google.com/github/siddheshtungare/Portfolio-analysis-streamlit/blob/main/Fundamental_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fundamentals Evaluation 

We will run some fundamentals comparison of how the tickers in our portfolio look against the other securities in the whole index

In [63]:
import yfinance as yf
import pandas as pd
import json
import hvplot.pandas

In [64]:
index_to_run = "^AXJO"

## Step1: Get data for all securities for that index. We have stored them in the `Resources/symbols_list.json` file

In [65]:
# Get all tickers for this index
with open('Resources/symbols_list.json', 'r') as json_file:
    symbols_dict = json.load(json_file)

tickers_names = symbols_dict[index_to_run]
# The symbols are read in this format: "NAB.AX: National Australia Bank". 
# We will need to split at the : and take the first part as our ticker codes
tickers_all = [ticker.split(":")[0] for ticker in tickers_names]

In [66]:
# list_of_tickers = ['MSFT', 'AAPL']

string_of_names = ""
for ticker in tickers_all:
    string_of_names += ticker + " "

string_of_names = string_of_names[:-1]
len(string_of_names)

1399

In [67]:
tickers = yf.Tickers(string_of_names)

Downloading data takes around 1-2 mins

In [80]:
# Download Fundamentals data for all the tickers
dict_info = [tickers.tickers[ticker].info for ticker in tickers_all]
len(dict_info)

200

In [81]:
list_of_keys = ["overallRisk", "averageVolume", "beta", "marketCap", "ebitda", "forwardPE", "totalRevenue" ]

In [82]:
whatisit = []
for index in range(len(tickers_all)):
    records = {"index": tickers_all[index]}
    records['overallRisk'] = dict_info[index].get('overallRisk')
    records['averageVolume'] = dict_info[index].get('averageVolume')
    records['beta'] = dict_info[index].get('beta')
    records['marketCap'] = dict_info[index].get('marketCap')
    records['ebitda'] = dict_info[index].get('ebitda')
    records['forwardPE'] = dict_info[index].get('forwardPE')
    records['totalRevenue'] = dict_info[index].get('totalRevenue')

    whatisit += [records]

df_fundamentals = pd.DataFrame(whatisit)

In [83]:
print(df_fundamentals.shape)
df_fundamentals.sample()

(200, 8)


,index,overallRisk,averageVolume,beta,marketCap,ebitda,forwardPE,totalRevenue
197,UMG.AX,3.0,1756470.0,0.627058,1.471961e+09,62300000.0,25.894737,1.514000e+09


### Data Cleanup 

There maybe some NaNs and some more nasty surprises

Check NaNs

In [108]:
df_fundamentals.isna().sum()

index            0
overallRisk      0
averageVolume    0
beta             0
marketCap        0
ebitda           0
forwardPE        0
totalRevenue     0
dtype: int64

NaNs present, we'll need to fill them up with some default values: using the mean of the columns values

In [85]:
for col in list_of_keys: 

    try: 
        df_fundamentals[col].fillna(df_fundamentals[col].mean(), inplace = True)
    except TypeError as err: 
        print(f"Exception TypeError raised for {col}")

df_fundamentals.isna().sum()

Exception TypeError raised for forwardPE


index            0
overallRisk      0
averageVolume    0
beta             0
marketCap        0
ebitda           0
forwardPE        8
totalRevenue     0
dtype: int64

Check dtypes

In [86]:
df_fundamentals.dtypes

index             object
overallRisk      float64
averageVolume    float64
beta             float64
marketCap        float64
ebitda           float64
forwardPE         object
totalRevenue     float64
dtype: object

`forwardPE` shouldnt be an Object. This means there are some junk values in there

In [87]:
df_fundamentals.forwardPE.describe()

count          192
unique         191
top       Infinity
freq             2
Name: forwardPE, dtype: object

In [96]:
df_fundamentals.loc[df_fundamentals.forwardPE == "Infinity"]

,index,overallRisk,averageVolume,beta,marketCap,ebitda,forwardPE,totalRevenue


**LOL** Price to Earnings ratio = Infinity 🤣

Set the forwardPE ratio to the mean of the column

Then replace the null values with the mean of the column 

In [93]:
mean_forwardPR = df_fundamentals.loc[df_fundamentals.forwardPE != "Infinity", "forwardPE"].mean()

In [97]:
df_fundamentals.loc[df_fundamentals.forwardPE == "Infinity", "forwardPE"] = mean_forwardPR
df_fundamentals['forwardPE'].fillna(df_fundamentals['forwardPE'].mean(), inplace = True)

Verify

In [98]:
df_fundamentals.forwardPE.describe()

count    200.000000
mean      17.618580
std       29.440167
min     -129.000000
25%       11.535298
50%       14.801282
75%       20.188645
max      320.571440
Name: forwardPE, dtype: float64

#### Data Cleanup complete

Store the cleaned data to a CSV, so that streamlit app can load the DF from it and display the plots

In [110]:
df_fundamentals.to_csv(f"Resources/Fundamentals_data_{index_to_run}.csv", index=None)

## Start exploring some plots

In [104]:
list_of_keys

['overallRisk',
 'averageVolume',
 'beta',
 'marketCap',
 'ebitda',
 'forwardPE',
 'totalRevenue']

In [111]:
plot = df_fundamentals.hvplot.kde("marketCap", title="Normal Distribution")

plot

:Distribution   [marketCap]   (Density)

In [115]:
plot = df_fundamentals.hvplot.kde("marketCap", title="Normal Distribution")

plot


:Distribution   [marketCap]   (Density)

In [131]:
import plotly.figure_factory as ff
import plotly.express as px
import numpy as np
import pandas as pd

fig = ff.create_distplot(
    [df_fundamentals["ebitda"].tolist()],
    group_labels=["ebitda"],
    show_hist=False,
).add_traces(
    px.histogram(df_fundamentals, x="ebitda")
    .update_traces(yaxis="y3", name="histogram")
    .data
).update_layout(yaxis3={"overlaying": "y", "side": "right"}, showlegend=False)

fig


In [123]:
import plotly.express  as px

px.histogram(df_fundamentals, x="averageVolume", histnorm='probability density')


In [43]:
box_plot = df_fundamentals.hvplot.box(y=['totalRevenue', 'marketCap'], width=500, height=400) #by='totalRevenue', 
box_plot

:BoxWhisker   [Variable]   (value)

## New plan - Draw a spider web (radar-chart)

y-axis = list_of_keys

x-axis = percentile score of a security

[https://plotly.com/python/radar-chart/](https://plotly.com/python/radar-chart/)